In [ ]:
import sys
import logging

import numpy as np

from math import sqrt, exp

In [ ]:
logging.basicConfig(
    stream=sys.stdout, 
    level=logging.DEBUG, 
    format='%(asctime)s %(levelname)s:%(name)s:%(message)s'
)

In [ ]:
def calc_f(r):
    def calc_c(r):
        return (1 + sqrt(1 - r ** 2)) * exp(-sqrt(1 - r ** 2))
    
    return sqrt(2.0 * calc_c(r) / exp(1))


def calc_r(k_1_prime, k_m1_prime, s_prime):
    return sqrt( 
        ((k_1_prime + k_m1_prime) - (k_1_prime - k_m1_prime) ** 2) / (2 * (k_1_prime + k_m1_prime - s_prime))
    )


def min_scalar_fraction(k_m1_prime, k_0_prime, k_1_prime):
    return max(2 * k_m1_prime - 1.0, 0.0) - 2 * min(k_m1_prime, k_1_prime) 


EPSILON = 1e-6


# Needed to describe the behavior of the logarithm of the binomial coefficient binom(alpha * n, beta * n).
def calc_base(alpha, beta):
    def equal(x, y):
        return (abs(x - y) < EPSILON)
    
    if equal(beta, 0.0):
        return 1.0

    if equal(alpha, beta):
        return 1.0

    return (alpha ** alpha) / float(beta ** beta) / float((alpha - beta) ** (alpha - beta))

In [ ]:
UPPER_BASE_BOUND = sqrt(2.0 / exp(1))

In [ ]:
def blocking_two_parted_construction_found(
    k_m1_prime, 
    k_0_prime, 
    k_1_prime, 
    s_prime,
    vertex_set_base,
    step_size,
):
    # First part
    for alpha in np.arange(step_size, 1.0, step_size):
        # Second part
        beta = 1 - alpha

        # Minus ones in the first part 
        for A in np.arange(step_size, 1.0, step_size):
            # Minus ones in the second part 
            D = (k_m1_prime - alpha * A) / beta

            if D < 0:
                continue

            # Zeros in the first part
            for B in np.arange(step_size, 1.0, step_size):
                # Zeros in the second part
                E = (k_0_prime - alpha * B) / beta

                if E < 0:
                    continue

                # Ones in the first part
                C = 1 - A - B
                # Ones in the second part                
                F = 1 - D - E

                if C < 0 or F < 0:
                    continue

                if alpha * min_scalar_fraction(A, B, C) + beta * min_scalar_fraction(D, E, F) < s_prime:
                    continue

                independent_set_base = calc_base(alpha, alpha * A) * \
                                       calc_base(alpha * (1 - A), alpha * B) * \
                                       calc_base(beta, beta * D) * \
                                       calc_base(beta * (1 - D), beta * E)

                if independent_set_base / vertex_set_base > UPPER_BASE_BOUND:
                    return True
    return False

In [ ]:
def blocking_three_parted_construction_found(
    k_m1_prime, 
    k_0_prime, 
    k_1_prime, 
    s_prime,
    vertex_set_base,
    step_size,
):
    # The relative sizes of the parts are alpha, beta and gamma with alpha + beta + gamma = 1.
    # The subdivison between coordinate values -1, 0 and 1 is as follows:
    # The realtive sizes are A, B, C in the first part, with A + B + C = 1;
    # D, E, F in the second part; G, H, I in the third part.
    
    for alpha in np.arange(step_size, 1.0, step_size):
        for beta in np.arange(step_size, 1.0 - alpha, step_size):
            gamma = 1 - alpha - beta

            if gamma <= 0:
                continue

            for A in np.arange(step_size, 1, step_size):
                for B in np.arange(step_size, 1 - A, step_size):
                    for D in np.arange(step_size, (k_m1_prime - alpha * A) / beta, step_size):
                        G = (k_m1_prime - alpha * A - beta * D) / gamma
                        if G < 0:
                            continue
                        
                        for E in np.arange(step_size, 1 - D, step_size):
                            C = 1 - A - B
                            F = 1 - D - E                            
                            
                            H = (k_0_prime - alpha * B - beta * E) / gamma
                            if H < 0:
                                continue
                            
                            I = (k_1_prime - alpha * C - beta * F) / gamma
                            if I < 0:
                                continue
                            
                            if (
                                alpha * min_scalar_fraction(A, B, C) + 
                                beta * min_scalar_fraction(D, E, F) + 
                                gamma * min_scalar_fraction(G, H, I)
                            ) < s_prime:
                                continue

                            independent_set_base = calc_base(alpha, alpha * A) * \
                                                   calc_base(alpha * (1 - A), alpha * B) * \
                                                   calc_base(beta, beta * D) * \
                                                   calc_base(beta * (1 - D), beta * E) * \
                                                   calc_base(gamma, gamma * G) * \
                                                   calc_base(gamma * (1 - G), gamma * H)

                            if independent_set_base / vertex_set_base > UPPER_BASE_BOUND:
                                return True
                                
    return False

In [ ]:
def try_update(
    best_estimate, 
    best_params, 
    k_m1_prime, 
    k_0_prime, 
    k_1_prime,
    s_prime, 
    step_size_two_parted,
    step_size_three_parted,
):
    updated = False
    r = calc_r(k_1_prime, k_m1_prime, s_prime)
    
    if r >= 0.5 and r <= 1.0:
        f = calc_f(r)
        chi = 1 / f

        if chi > best_estimate:
            vertex_set_base = calc_base(1.0, k_m1_prime) * calc_base(1 - k_m1_prime, k_0_prime)
            
            two_parted_found = blocking_two_parted_construction_found(
                k_m1_prime=k_m1_prime, 
                k_0_prime=k_0_prime, 
                k_1_prime=k_1_prime, 
                s_prime=s_prime,
                vertex_set_base=vertex_set_base,
                step_size=step_size_two_parted,
            )
            
            if two_parted_found:
                logging.debug(
                    'A blocking two-parted construction found for params %s, %s, %s, %s',     
                    k_m1_prime, 
                    k_0_prime, 
                    k_1_prime,
                    s_prime,
                )
                return best_estimate, best_params, updated
            
            three_parted_found = blocking_three_parted_construction_found(
                k_m1_prime=k_m1_prime, 
                k_0_prime=k_0_prime, 
                k_1_prime=k_1_prime, 
                s_prime=s_prime,
                vertex_set_base=vertex_set_base,
                step_size=step_size_three_parted,
            )
            
            if three_parted_found:
                logging.debug(
                    'A blocking three-parted construction found for params %s, %s, %s, %s',     
                    k_m1_prime, 
                    k_0_prime, 
                    k_1_prime,
                    s_prime,
                )
                return best_estimate, best_params, updated
                        
            best_estimate = chi
            best_params = (k_m1_prime, k_0_prime, k_1_prime, s_prime)
            updated = True
    
    return best_estimate, best_params, updated

In [ ]:
DELTA_FRACTION = 0.05
DELTA_SCALAR = 0.01
USE_HISTORY = True

if USE_HISTORY:
    # Initializing by the results of the previous computations
    best_estimate = 1.28000340218
    best_params = (0.25, 0.25, -0.05585)
    DELTA_TWO_PARTED_CONSTRUCTION_CHECKS = 0.0125
    DELTA_THREE_PARTED_CONSTRUCTION_CHECKS = 0.03
    blank = True
else:
    best_estimate = 1.0
    best_params = None
    DELTA_TWO_PARTED_CONSTRUCTION_CHECKS = 0.015
    DELTA_THREE_PARTED_CONSTRUCTION_CHECKS = 0.1
    blank = False

for k_1_prime in np.arange(DELTA_FRACTION, 0.5, DELTA_FRACTION):
    for k_m1_prime in np.arange(DELTA_FRACTION, min(k_1_prime, 0.5 - k_1_prime), DELTA_FRACTION):
        k_0_prime = 1 - k_1_prime - k_m1_prime
        
        s_prime_lower = -2.0 * k_m1_prime
        s_prime_upper = (k_1_prime - k_m1_prime) / 2.0
        
        for s_prime in np.arange(s_prime_lower, s_prime_upper, DELTA_SCALAR):        
            best_estimate, best_params, updated = try_update(
                best_estimate=best_estimate, 
                best_params=best_params,
                k_m1_prime=k_m1_prime, 
                k_0_prime=k_0_prime, 
                k_1_prime=k_1_prime,
                s_prime=s_prime,
                step_size_two_parted=DELTA_TWO_PARTED_CONSTRUCTION_CHECKS,
                step_size_three_parted=DELTA_THREE_PARTED_CONSTRUCTION_CHECKS,                
            )
            if updated or blank:
                logging.info('base for chi_m >= {0}, (k_{{-1}}, k_{{0}}, k_{{1}}) = {1}'.format(
                    best_estimate, best_params
                ))
                blank = False